In [28]:
import pandas as pd
import polars as pl
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from scipy.stats import chi2_contingency
from sklearn.feature_selection import chi2
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance

In [3]:
df = pl.read_csv('../../Dementia/JanBDRcount.csv')
df = df.to_pandas()

In [4]:
for col in df.columns:
    df[col].fillna(3, inplace=True)

print(df.shape)
X = df

y = X['PHENOTYPE']
X = X.drop(columns=['FID', 'IID', 'PAT', 'MAT', 'SEX', 'PHENOTYPE'])

(534, 297684)


In [5]:
assert X.isnull().sum().sum() == 0, "There are still missing values in X"
assert y.isnull().sum().sum() == 0, "There are still missing values in y"

In [6]:
print(X.isnull().sum().sum())

0


In [12]:
# Number of features per chunk
features_per_chunk = 1000
num_features = X.shape[1]
# Number of chunks
num_chunks = num_features // features_per_chunk
print(num_chunks)
print(X.shape[1])
# Split the data into chunks
chunks = []
for i in range(num_chunks):
    start_index = i * features_per_chunk
    if i == num_chunks:
        # Ensure the last chunk includes all remaining features
        end_index = num_features
    else:
        end_index = (i + 1) * features_per_chunk
    chunk = X.iloc[:, start_index:end_index]
    chunks.append(chunk)

chunks.append(X.iloc[:, num_chunks*features_per_chunk:])

# Check the shape of the first few chunks to verify
# for i, chunk in enumerate(chunks[:]):
#     chunk_with_target = pd.concat([chunk, y], axis=1)
#     chunk_with_target.to_csv(f'chunk_{i + 1}.csv', index=False)
#     print(f'Saved chunk_{i + 1}.csv')
    


297
297678


In [25]:
# Perform logistic regression and select best features
top_n_features = 20  # For demonstration, selecting top 2 features
best_features = []

for i, chunk in enumerate(chunks):
    chunk_with_target = pd.concat([chunk, y], axis=1)
    X_chunk = chunk_with_target.drop(columns=['PHENOTYPE'])
    y_chunk = chunk_with_target['PHENOTYPE']
    model = LogisticRegression(max_iter=10000)
    model.fit(X_chunk, y_chunk)
    coef_abs = np.abs(model.coef_[0])
    top_features_idx = np.argsort(coef_abs)[-top_n_features:]
    top_features = X_chunk.columns[top_features_idx]
    best_features.extend(top_features)
    print(f"Chunk {i + 1} - Selected Features: {top_features.tolist()}")


Chunk 1 - Selected Features: ['rs7539775_G', 'rs12563394_G', 'rs742359_A', 'rs10909901_A', 'rs7527871_C', 'rs2821060_A', 'rs2817168_G', 'rs819980_G', 'rs7515488_A', 'rs845246_A', 'rs7340016_G', 'rs2485945_G', 'rs6663882_G', 'rs12564077_G', 'rs6669278_A', 'rs2651899_G', 'rs170551_G', 'rs2341354_A', 'rs2142569_A', 'rs6698548_A']
Chunk 2 - Selected Features: ['rs867321_A', 'rs9430624_A', 'rs7547591_G', 'rs12758112_A', 'rs12739601_G', 'rs753305_G', 'rs9728569_A', 'rs2977227_G', 'rs17436172_A', 'rs2095842_G', 'rs10803369_C', 'rs555115_A', 'rs4384275_A', 'rs3856278_G', 'rs109490_A', 'rs6689677_C', 'rs11807837_A', 'rs542634_A', 'rs17036032_G', 'rs1280971_A']
Chunk 3 - Selected Features: ['rs616145_A', 'rs6684096_G', 'rs11261071_A', 'rs12030578_A', 'rs6677615_A', 'rs2301478_A', 'rs2745251_A', 'rs12565650_A', 'rs17433222_A', 'rs172378_G', 'rs1160680_G', 'rs2816025_A', 'rs4515757_A', 'rs7524076_G', 'rs11249245_G', 'rs292001_A', 'rs2229493_A', 'rs829391_G', 'rs6686713_C', 'rs2796354_A']
Chunk 4 -

In [26]:
print(best_features)

['rs7539775_G', 'rs12563394_G', 'rs742359_A', 'rs10909901_A', 'rs7527871_C', 'rs2821060_A', 'rs2817168_G', 'rs819980_G', 'rs7515488_A', 'rs845246_A', 'rs7340016_G', 'rs2485945_G', 'rs6663882_G', 'rs12564077_G', 'rs6669278_A', 'rs2651899_G', 'rs170551_G', 'rs2341354_A', 'rs2142569_A', 'rs6698548_A', 'rs867321_A', 'rs9430624_A', 'rs7547591_G', 'rs12758112_A', 'rs12739601_G', 'rs753305_G', 'rs9728569_A', 'rs2977227_G', 'rs17436172_A', 'rs2095842_G', 'rs10803369_C', 'rs555115_A', 'rs4384275_A', 'rs3856278_G', 'rs109490_A', 'rs6689677_C', 'rs11807837_A', 'rs542634_A', 'rs17036032_G', 'rs1280971_A', 'rs616145_A', 'rs6684096_G', 'rs11261071_A', 'rs12030578_A', 'rs6677615_A', 'rs2301478_A', 'rs2745251_A', 'rs12565650_A', 'rs17433222_A', 'rs172378_G', 'rs1160680_G', 'rs2816025_A', 'rs4515757_A', 'rs7524076_G', 'rs11249245_G', 'rs292001_A', 'rs2229493_A', 'rs829391_G', 'rs6686713_C', 'rs2796354_A', 'rs16835742_A', 'rs3845473_G', 'rs9659735_A', 'rs6700641_A', 'rs489408_A', 'rs10914446_A', 'rs7867

In [29]:
# Number of top features to select from each chunk
top_n_features = 20

# List to store the best features from each chunk
best_features = []

# Hyperparameter grid for logistic regression
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']  # Using liblinear as it supports l1 and l2 penalties
}

for i, chunk in enumerate(chunks):
    # Append the target column
    chunk_with_target = pd.concat([chunk, y], axis=1)

    # Split into features and target
    X_chunk = chunk_with_target.drop(columns=['PHENOTYPE'])
    y_chunk = chunk_with_target['PHENOTYPE']

    # Logistic Regression model
    logreg = LogisticRegression(max_iter=10000)

    # Grid Search CV for hyperparameter tuning
    grid_search = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_chunk, y_chunk)

    # Best model from grid search
    best_model = grid_search.best_estimator_

    # Get absolute values of the coefficients
    coef_abs = np.abs(best_model.coef_[0])

    # Get indices of top N features
    top_features_idx = np.argsort(coef_abs)[-top_n_features:]

    # Get the names of the top N features
    top_features = X_chunk.columns[top_features_idx]
    best_features.extend(top_features)

    # Print selected features for each chunk
    print(f"Chunk {i + 1} - Selected Features: {top_features.tolist()}")


Chunk 1 - Selected Features: ['rs10158288_G', 'rs2821023_A', 'rs11584295_A', 'rs742359_A', 'rs228642_G', 'rs6697749_A', 'rs17030082_A', 'rs6669278_A', 'rs12757620_C', 'rs2382832_A', 'rs16840838_A', 'rs17032760_A', 'rs6698548_A', 'rs2651899_G', 'rs2341354_A', 'rs7545284_A', 'rs3813199_A', 'rs12564077_G', 'rs2485945_G', 'rs4648377_A']
Chunk 2 - Selected Features: ['rs703801_G', 'rs10803369_C', 'rs1148460_A', 'rs11807837_A', 'rs12404333_G', 'rs6665486_G', 'rs4908853_A', 'rs7550295_A', 'rs4662139_G', 'rs10492987_A', 'rs109490_A', 'rs12739601_G', 'rs3737964_A', 'rs6689677_C', 'rs9430631_A', 'rs17036032_G', 'rs374450_A', 'rs1280971_A', 'rs41268336_A', 'rs145264193_0']
Chunk 3 - Selected Features: ['rs10917420_G', 'rs12567861_A', 'rs4649002_G', 'rs10903113_A', 'rs11247933_A', 'rs7535656_A', 'rs3893315_G', 'rs617367_A', 'rs4912036_A', 'rs12122703_A', 'rs12145714_A', 'rs11587947_A', 'rs292001_A', 'rs7544348_G', 'rs4655059_C', 'rs6686713_C', 'rs11261071_A', 'rs2796354_A', 'rs9658850_A', 'rs10916

In [30]:
# Number of top features to select from each chunk
top_n_features = 20

# List to store the best features from each chunk
best_features = []

# Hyperparameter grid for logistic regression
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']  # Using liblinear as it supports l1 and l2 penalties
}

for i, chunk in enumerate(chunks):
    # Append the target column
    chunk_with_target = pd.concat([chunk, y], axis=1)

    # Split into features and target
    X_chunk = chunk_with_target.drop(columns=['PHENOTYPE'])
    y_chunk = chunk_with_target['PHENOTYPE']

    # Logistic Regression model
    logreg = LogisticRegression(max_iter=10000)

    # Grid Search CV for hyperparameter tuning
    grid_search = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_chunk, y_chunk)

    # Best model from grid search
    best_model = grid_search.best_estimator_

    # Calculate permutation importance
    perm_importance = permutation_importance(best_model, X_chunk, y_chunk, n_repeats=10, random_state=0)
    perm_importances = perm_importance.importances_mean

    # Get indices of top N features based on permutation importance
    top_features_idx = np.argsort(perm_importances)[-top_n_features:]

    # Get the names of the top N features
    top_features = X_chunk.columns[top_features_idx]
    best_features.append(top_features)

    # Print selected features for each chunk
    print(f"Chunk {i + 1} - Selected Features: {top_features.tolist()}")


Chunk 1 - Selected Features: ['rs707582_G', 'rs12735231_A', 'rs6697749_A', 'rs9286971_G', 'rs17032760_A', 'rs7340016_G', 'rs12044274_A', 'rs161799_G', 'rs11584295_A', 'rs12058103_G', 'rs17030082_A', 'rs12141954_G', 'rs6669278_A', 'rs6698548_A', 'rs16840838_A', 'rs7545284_A', 'rs12564077_G', 'rs12757620_C', 'rs2485945_G', 'rs4648377_A']
Chunk 2 - Selected Features: ['rs9430631_A', 'rs552749_A', 'rs630075_A', 'rs615917_G', 'rs7550969_A', 'rs10803369_C', 'rs1721829_A', 'rs7520597_C', 'rs7547591_G', 'rs12725198_A', 'rs11807837_A', 'rs10492987_A', 'rs4908812_A', 'rs7550295_A', 'rs4908853_A', 'rs109490_A', 'rs6696295_G', 'rs4662139_G', 'rs17036032_G', 'rs41268336_A']
Chunk 3 - Selected Features: ['rs2229493_A', 'rs7523920_A', 'rs6686713_C', 'rs4912013_G', 'rs10917420_G', 'rs10492996_A', 'rs292001_A', 'rs7531434_A', 'rs12567861_A', 'rs2796354_A', 'rs12145714_A', 'rs617367_A', 'rs4912036_A', 'rs11587947_A', 'rs12122703_A', 'rs11261071_A', 'rs9658850_A', 'rs3893315_G', 'rs4655059_C', 'rs7544348